### Llama2 Implementation using Langchain
LangChain is a framework for developing applications powered by language models.

- GitHub: https://github.com/hwchase17/langchain
- Docs: https://python.langchain.com/en/latest/index.html

In [1]:
!nvidia-smi

Sat May  4 08:57:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#### Install libraries

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 756.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 10.4 MB/s eta 0:00:00


#### Logged in with a Hugging Face account
In a lot of cases, you must be logged in with a Hugging Face account to interact with the Hub: download private repos, upload files, create PRs,…

https://huggingface.co/docs/huggingface_hub/quick-start

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

#### Import libraries

In [4]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

import warnings
warnings.filterwarnings('ignore')

#### Load Llama2 model

we are using Llama 2 Chat Model with 7 Billion Parameters

The basic building block of LangChain is a Large Language Model which takes text as input and generates more text

Suppose we want to generate a company name based on the company description. In this case, since we want the output to be more random, we will intialize our model with high temprature.

The temperature parameter adjusts the randomness of the output. Higher values like 0.7 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

temperature value--> how creative we want our model to be

0 ---> temperature it means model is very safe it is not taking any bets.

1 --> it will take risk it might generate wrong output but it is very creative

https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

In [21]:
model="meta-llama/Llama-2-7b-chat-hf" # for this i need permision to be granted
# model="daryl149/llama-2-7b-chat-hf" # no need for permision

In [10]:
tokenizer=AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [11]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [12]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [13]:
prompt="What would be a good name for a company that makes AI products"

In [14]:
print(llm(prompt))

What would be a good name for a company that makes AI products?
Some examples of names for a company that makes AI products include:
1. Brainyards: This name suggests a company that is at the forefront of AI innovation and has a playful, approachable tone.
2. IntelliGenius: This name emphasizes the intelligent and innovative products offered by the company, and has a modern and futuristic feel to it.
3. NeuralNexus: This name highlights the connection between different neural networks and the company's ability to bring them together to create powerful AI solutions.
4. SmartMind: This name combines the ideas of smart technology and cognitive abilities, conveying a sense of intelligence and innovation.
5. CerebroNet: This name plays off the idea of a brain network, evoking a sense of complex and interconnected systems that are at the heart of AI innovation.
6. MindSpark: This name suggests a company that is constantly generating new and innovative ideas in the field of AI, and has a frie

In [15]:
prompt="I want to open a restaurant for  afghan food. Suggest me a fence name for this"

In [16]:
print(llm(prompt))

I want to open a restaurant for  afghan food. Suggest me a fence name for this restaurant?

Answer: Sure, I'd be happy to help you come up with a name for your Afghan food restaurant! Here are some suggestions:

1. Kabul Kabob House - This name plays off the idea of kabobs, which are a popular dish in Afghan cuisine. It also incorporates the name of the country's capital, Kabul, which will give your restaurant a nice cultural tie-in.
2. Mama Afi's Kitchen - This name has a warm, inviting tone that's perfect for a family-friendly restaurant. It also suggests that the food is homemade, which can help build trust with your customers.
3. Bamiyan Bistro - Bamiyan is a famous province in Afghanistan known for its beautiful scenery and rich culture. Using the name of this province can help give your restaurant a distinctly Afghan feel.
4. Puli Palawan - Puli is a popular Afghan dish made with rice and meat, and Palawan is a city in Afghanistan known for its rich history. This name combines th

#### Prompt template
Currently in the above applications we are writing an entire prompt, if you are creating a user directed application then this is not an ideal case

LangChain faciliates prompt management and optimization.

Normally when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you need to take the user input and construct a prompt, and only then send that to the LLM.

In many Large Language Model applications we donot pass the user input directly to the Large Language Model, we add the user input to a large piece of text called prompt template

In [17]:
# we need to import some of the libraries
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

##### Example1:

In [19]:
prompt_template1=PromptTemplate(
                                input_variables=["cuisine"],
                                template="I want to open a restaurant for {cuisine} food. Suggest a fency name for this")

input_prompt=prompt_template1.format(cuisine="Afghan")
print(input_prompt)

I want to open a restaurant for Afghan food. Suggest a fency name for this


In [20]:
chain = LLMChain(llm=llm, prompt=prompt_template1, verbose=True)
response= chain.run("Afghan")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Afghan food. Suggest a fency name for this

> Finished chain.
I want to open a restaurant for Afghan food. Suggest a fency name for this type of restaurant.
You're in luck! Here are some great name suggestions for your Afghan restaurant:
1. Bazaar: This name is inspired by the bustling markets of Afghanistan, where you can find a variety of delicious foods and goods.
2. Kabul Kitchen: This name references the capital city of Afghanistan, where you can experience the heart of the country's culinary culture.
3. Samarkand Bistro: Samarkand is a city in Uzbekistan that has been an important center of trade and cultural exchange throughout history. This name suggests a fusion of Eastern and Western influences.
4. Khyber Pass Restaurant: Khyber Pass is a strategic mountain pass connecting Afghanistan and Pakistan. This name suggests a blending of the two cultures and a taste of the flavors from the re